# <u>Web Scraping</u>

## <u>Part \#2: Web Scraping and Web Crawling Using *Beautiful Soup*</u>

The World Wide Web is a potential treasure trove for a data scientist. But in order to access this data, we need to find it and extract it. In this notebook, you we learn about this process by web scraping and web crawling using a Python module called **Beautiful Soup**.

As an introduction to web scraping we will start by opening a page on Wikipedia, find all links to other Wikipedia pages on that page, then pick one of those links (randomly) to 'crawl' to another page.  We will continue this process and see where it leads us.

### <u>Importing Useful Modules</u>

First we will need to access modules to assist us - some of them are familiar, others are new.  Add some code comments to explain what these imports are used for.  You do not need to say exactly what we will use them for, just what their general purpose is.  You may want to look this up on the Internet.

In [44]:
from urllib.request import urlopen           # Used to open URL webpages
from bs4 import BeautifulSoup                # Used to read HTML of webpages 
import datetime                              # Used to create/use date and time objects
import random                                # Used to generate random numbers
import re                                    # Used to compare if Strings to expressions
import time                                  # Used to create time objects
from IPython.display import Image, display   # Used to display images in Python Notebook

### <u>Get Links from a Wikipedia Page</u>

Let's see how many links to other Wikipedia pages we can find on the Wikipedia page for Fremd High School.  First we will define a general function (method) to get links off of any Wikipedia page.  Remember that a big advantage of using functions (methods) is that we can reuse code and perform the same task over and over with different inputs but only one group of code statements. In short, this is an **abstraction that helps manage the complexity of our program**. 

In our case here we will be able to use this function to find page links for any Wikipedia page and not just FHS's page.  This will make it possible (and much more efficient) to perform our random 'crawl' through Wikipedia pages.    

In [45]:
def get_links(article_url):                                     # Create function to extract links from a given Wiki page
    html_page = urlopen("http://en.wikipedia.org"+article_url)  # Opens whatever page we are requesting
    bs_obj = BeautifulSoup(html_page, 'html.parser')            # Saves the html in a Beautiful Soup object
    
    # The next line finds specific HTML elements in the page we opened - notice the familiar tags and properties
    return bs_obj.find("div",{"id":"bodyContent"}).findAll("a",href=re.compile("^(/wiki/)((?!:).)*$"))

*Notice:* The return statement above uses a *regular expression*. The *re.compile()* method puts together an expression for the *findAll()* method to match to a certain *href*. The text below briefly explains what the various parts of the regular expression mean. 

What does *^(/wiki/)* look for?
    - The start of a string followed by the literal "/wiki/"
    
What does _((?!:).)*_ look for?
    - As many occurences as exist of anything that is not a ":" followed by a "."
    
What does  _$_ look for?
    - Anything from the current location to the end of the string

### <u>Counting the Links on Fremd's Wikipedia Page</u>

Next we will call our function, *get_links(articleUrl)*, by sending in the name of Wikipedia's Fremd High School page. The entire url is https://en.wikipedia.org/wiki/William_Fremd_High_School but we only need to send in the last part of the url since that is how we wrote our function.

In [46]:
links=get_links("/wiki/William_Fremd_High_School")  # Call to function created above

**<u>Question 4:</u>** How many links are on the Fremd High School Wikipedia page? You will need to write code in the cell below to find out.

*Hint: 'links' is a list, how can you find out how long it is?*
    
**<u>Your Answer: 276 lil bro.</u>** 

In [47]:
# Your code here
len(links)

276

**<u>Question 5:</u>**  What type of data is stored in links? You will need to write code in the cell below to find out.

**<u>Your Answer:</u>** 

In [48]:
# Your code below
print(type(links))

<class 'bs4.element.ResultSet'>


### <u>View the Links Stored in the Variable '*links*'</u>

Now let's look at all of these links:

In [49]:
print(links)

[<a href="/wiki/Palatine,_Illinois" title="Palatine, Illinois">Palatine</a>, <a href="/wiki/Illinois" title="Illinois">Illinois</a>, <a href="/wiki/Geographic_coordinate_system" title="Geographic coordinate system">Coordinates</a>, <a href="/wiki/Township_High_School_District_211" title="Township High School District 211">Township H.S. 211</a>, <a href="/wiki/Full-time_equivalent" title="Full-time equivalent">FTE</a>, <a href="/wiki/Forest_green" title="Forest green">Forest Green</a>, <a href="/wiki/Gold_(color)" title="Gold (color)">Gold</a>, <a href="/wiki/Fight_song" title="Fight song">Fight song</a>, <a href="/wiki/Mid-Suburban_League" title="Mid-Suburban League">Mid-Suburban League</a>, <a class="mw-redirect" href="/wiki/High_school" title="High school">high school</a>, <a href="/wiki/Palatine,_Illinois" title="Palatine, Illinois">Palatine, Illinois</a>, <a class="mw-redirect" href="/wiki/Chicago,_Illinois" title="Chicago, Illinois">Chicago, Illinois</a>, <a href="/wiki/United_Sta

### <u>Clean the List</u>

Now let's clean the page links up a bit. First we will use a list comprehension to create a list of only the href portion of each tag. Then we will look only at the page names by removing the text that appears at the beginning of each link.  

In [50]:
link_list=[link['href'] for link in links]   # Creates a list of href's for every link in the list
for link in link_list:                       # Iterate over the newly formed list
    link = link.replace('/wiki/','')         # Remove the repetitive text
    print(link)                              # Display the page names one per line

Palatine,_Illinois
Illinois
Geographic_coordinate_system
Township_High_School_District_211
Full-time_equivalent
Forest_green
Gold_(color)
Fight_song
Mid-Suburban_League
High_school
Palatine,_Illinois
Chicago,_Illinois
United_States
Township_High_School_District_211
James_B._Conant_High_School
Hoffman_Estates_High_School
Palatine_High_School
Schaumburg_High_School
Newsweek
U.S._News_%26_World_Report
Union_Pacific_Northwest_Line
Hoffman_Estates
Rolling_Meadows
Schaumburg,_Illinois
South_Barrington
Inverness,_Illinois
Plum_Grove_Junior_High_School
Illinois
Palatine_High_School
Minnesota_Vikings
Artificial_turf
The_New_York_Times
Township_High_School_District_211_transgender_student_incidents
SAT
ACT_(examination)
United_States_Department_of_Education
Newsweek
U.S._News_%26_World_Report
Mid-Suburban_League
Illinois_High_School_Association
Viking
Basketball
Cross_country_running
Golf
Gymnastics
Soccer
Swimming_(sport)
Diving_(sport)
Tennis
Track_and_field
Volleyball
Chess
Water_polo
Basebal

### <u>Web Crawling</u>

Enough of just looking at the Fremd HS Wikipedia page. Now let's try to 'crawl' through some pages by randomly picking one of the links on the Fremd High School page and following it to see what page links appear on that page. Then we will pick a random link on the new page and continue this process until we have 20 links from 20 pages we've crawled through.

In [51]:
import requests
from IPython.display import Image, display

#This next line "seeds" the random number generator at the current time (gives a somewhat random starting point)
random.seed(int(datetime.datetime.now().strftime('%f')))  

links=get_links("/wiki/William_Fremd_High_School")                      # Extracts and creates a list of links from Fremd's webpage
for i in range(20):                                                     # Loop to run 20 times
    new_article = links[random.randint(0,len(links)-1)].attrs['href']   # Chooses random link from the list of links
    print(new_article)                                                  # Prints the random link
    links = get_links(new_article)                                      # Extract and creates a list of links from the new webpage
    time.sleep(1)                                                       # 1-second break before another loop

/wiki/Mid-Suburban_League
/wiki/Wheeling,_Illinois
/wiki/Elk_Grove_Township,_Cook_County,_Illinois
/wiki/Park_Ridge,_Illinois
/wiki/Maine_East_High_School
/wiki/United_States_Court_of_Appeals_for_the_Eighth_Circuit
/wiki/Minnesota
/wiki/Iron_Range
/wiki/St._Louis_County,_Minnesota
/wiki/Laurentian_Divide
/wiki/Nelson_River
/wiki/Lake_Skokomish
/wiki/Glacial_Lake_Russell
/wiki/Skokomish_River
/wiki/December_2007_Pacific_Northwest_storms
/wiki/Great_Gale_of_1880
/wiki/Fort_William_(Oregon)
/wiki/Asa_Lovejoy
/wiki/Oregon_missionaries
/wiki/Ezra_Meeker


One of the reasons we picked Wikipedia pages to crawl through is that by doing web scraping there is the potential to flood a web server with requests. Wikipedia is used to receiving a lot of requests and can generally handle the volume. It is good practice, though, and considered common courtesy to put code into your web scraping program to pause in between requests.  The code is a simple "sleep" request. You will see this above with the link of code that reads *time.sleep(1)*.

### <u>Scraping Images</u>

We can scrape many kinds of data from webpages. Instead of just links, let's try scraping images. First, here's a function that looks for the URL of the main image for a Wikipedia article:

In [52]:
def get_image_url(article_url):
    html_page = urlopen("http://en.wikipedia.org"+article_url)                    # Opens whatever page we are requesting
    bs_obj = BeautifulSoup(html_page, 'html.parser')                              # Saves the html in a Beautiful Soup object
    try:
        image_url = bs_obj.find("meta",{"property":"og:image"}).attrs['content']  # Finds fist image on webpage, stores URL
    except AttributeError:
        image_url = False                                                         # If no URL is available, store 'False' instead
    return image_url                                                              # Return image's URL

Test it out with the Fremd High School Wikipedia page:

In [53]:
get_image_url("/wiki/William_Fremd_High_School")  # Call to function created above

'https://upload.wikimedia.org/wikipedia/en/1/13/William_Fremd_High_School_Logo.png'

Now a function that takes a Wikipedia article URL as input and then performs the following steps:

* Get the URL of the main image on the page
* Saves the image as *output_image*
* Display the image in the notebook

In [54]:
def return_image(wiki_url,image_width):                         # Create function that accepts a Wiki topic and width desired
    
    if get_image_url(wiki_url) != False:                        # If there is an available URL        
        headers = {'User-Agent': 'Mozilla/5.0'}                     # Add a user-agent to our request
        img=get_image_url(wiki_url)                                 # Extract image's URL
        request = requests.get(img, headers = headers)              # Request the image
        if request.status_code == 200:                              # If the request resolves "Ok"
            url_to_file = request.content                           # Extract image data from URL
            extension = img.split('.')[-1]                          # Extract image's name
            name = "output/output_image." + extension               # Create new name for image file
            with open(name, 'wb') as image:                         # Create a new file to write information to
                image.write(url_to_file)                            # Write the images data to the file
            display(Image(filename=name,width=image_width))         # Open image in notebook
        else:                                                   # Otherwise, give the status code
            print("Error, Status Code:", request.status_code)

Let's test *return_image(wiki_url,image_width)* on the Fremd High School Wikipedia page and set an image width of 200 pixels:

In [55]:
return_image("/wiki/William_Fremd_High_School", 200)  # Call to function created above

FileNotFoundError: [Errno 2] No such file or directory: 'output/output_image.png'

Go Vikings! Now Let's repeat our web crawl, but now with images (when available). 
* *Notes:*
    * Remember that this is a random crawl, so it is possible that you could encounter content on Wikipedia that is not school-appropriate. If this happens, simply rerun the cell below:
    * Some links may just display text, while others will also display an image (if present)

In [ ]:
# This next line "seeds" the random number generator at the current time (gives a somewhat random starting point)
random.seed(int(datetime.datetime.now().strftime('%f')))  

links=get_links("/wiki/William_Fremd_High_School")                      # Extracts and creates a list of links from Fremd's webpage
for i in range(20):                                                     # Loop to run 20 times
    new_article = links[random.randint(0,len(links)-1)].attrs['href']   # Chooses random link from the list of links
    return_image(new_article,100)                                       # Call function created above, displays first image from link
    print(new_article)                                                  # Prints the random link
    links = get_links(new_article)                                      # Extracts and creates a list of links from the new webpage
    time.sleep(1)                                                       # 1-second break before another loop

### <u>Experiment With a New Wikipedia Page</u>

Now pick a Wikipedia page on a school-appropriate topic you are interested in and then try the following: 

#### <u>Scrape URL links from this page</u> 
   * Create a 'cleaned up' list of all the links from this wikipage

In [56]:
# Your code here
links=get_links("https://en.wikipedia.org/wiki/SpaceX")  # Call to function created above

URLError: <urlopen error [Errno 11001] getaddrinfo failed>

#### <u>Scrape an image from this page</u>
* Find the main picture from this Wikipedia page and display it in this notebook

In [ ]:
# Your code here


#### <u>Start your own web crawl</u>
* Run code to crawl through pages based on links that are found on the wiki pages
* Start with your chosen wiki page, then run a loop 20 times to open a random link from each new webpage
* Display all links and images found

*Hints:*
   * Only one change is needed from the code we used above to crawl from FHS's wiki! 
   * Change 'links' to be your list of links you created just above

In [ ]:
# Your code here


### <u>Answer the Following Wrap-up Questions:</u>  

**<u>Question 6:</u>**  What is web scraping? 

*Hint: Read https://en.wikipedia.org/wiki/Web_scraping*

**<u>Your Answer:</u>** 

**<u>Question 7:</u>**  What is web crawling? 

*Hint: Read https://en.wikipedia.org/wiki/Web_crawler*

**<u>Your Answer:</u>** 

**<u>Question 8:</u>** What is the difference between web scraping and web crawling?  

**<u>Your Answer:</u>** 
